In [2]:
import torch 
import torchvision 
import torch.nn as nn 

In [4]:
efficientNet_lar = torchvision.models.efficientnet_v2_m()
# print(efficientNet_lar)

In [12]:
import torch.nn as nn 
import torch 
import torchvision 

efficientnet = torchvision.models.efficientnet_v2_m()

model = efficientnet.features
e_avg_pool = efficientnet.avgpool
e_classifier = efficientnet.classifier

classifier_in_features = e_classifier[1].in_features

print(classifier_in_features)

children_list = list(model.children())
print(children_list[8])


from onnx_file import create_onnx_from_model

1280
Conv2dNormActivation(
  (0): Conv2d(512, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(1280, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): SiLU(inplace=True)
)


In [6]:
stage0 = nn.Sequential(
    children_list[0],
    children_list[1],
    children_list[2],
    children_list[3],
    children_list[4],
    nn.Sequential(*children_list[5][0:5])
)

inp_0 = torch.rand(16, 3, 32, 32)
create_onnx_from_model(stage0, "stage0.onnx", inp_0, 16)


[ONNX] Exporting ONNX model stage0.onnx
[ONNX] Exported to: stage0.onnx


In [7]:
stage1 = nn.Sequential(
    nn.Sequential(*children_list[5][5:]),
    nn.Sequential(*children_list[6][0:7]) # continue
)

inp_1 = stage0(inp_0)
create_onnx_from_model(stage1, "stage1.onnx", inp_1, 16)

print(stage1)

[ONNX] Exporting ONNX model stage1.onnx
[ONNX] Exported to: stage1.onnx
Sequential(
  (0): Sequential(
    (0): MBConv(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(176, 1056, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(1056, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(1056, 1056, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=1056, bias=False)
          (1): BatchNorm2d(1056, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
        )
        (2): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(1056, 44, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(44, 1056, kernel_size=(1, 1), stride=(1, 1))
          (activation): SiLU(inplace=True)
          (scale_activation): Sigmoid()
    

In [10]:
stage2 = nn.Sequential(
    nn.Sequential(*list(children_list[6][7:])),
    children_list[7],
    children_list[8], # Conv2dNormActivation
    e_avg_pool,
    nn.Flatten(1),
    nn.Dropout1d(p = 0.2, inplace=True),
    nn.Linear(in_features= classifier_in_features, out_features=10)
    # e_classifier
)

inp = torch.rand(16, 3, 32, 32)
in_1 = stage0(inp)
in_2 = stage1(in_1)
out_2 = stage2(in_2)

print(out_2.shape)

create_onnx_from_model(stage2, "stage2.onnx", in_2, 16)


torch.Size([16, 10])
[ONNX] Exporting ONNX model stage2.onnx
[ONNX] Exported to: stage2.onnx
